In [ ]:
import sys,os
import uproot4 as uproot
import awkward1 as ak
import json
import numpy as np
import matplotlib.pyplot as plt
from fcc_python_tools.locations import loc
from fcc_python_tools import kinematics
from particle import literals as lp
from fcc_python_tools import plotting
import tensorflow as tf
import zfit
import random

from matplotlib import rc
rc('font',**{'family':'serif','serif':['Roman']})
rc('text', usetex=True)

file_type = "p8_ee_Zcc_ecm91_EvtGen_D2PiPi0"

#Load 1 sub-file of 100k events
file = uproot.open(f"/eos/experiment/fcc/ee/generation/DelphesEvents/fcc_tmp/{file_type}/events_163459820.root")
tree = file['events']

#Get all the variables in the tree
vars = tree.keys()

#Loacation of the reco particles
r_container = "ReconstructedParticles"
r_c = f'{r_container}/{r_container}'

#Keep the variables that aren't covMatrix
keep_vars = []
for v in vars:
    if("covMatrix" not in v and r_c in v):
        keep_vars.append(v)

n_keep = 10000

r = tree.arrays(keep_vars,how="zip")[:n_keep]

In [ ]:
r[r_c,'p'] = kinematics.calc_p(r,r_c)
p_cut = r[r_c,"p"] > 0.
r = r[p_cut]

#Pions
pi_cut = abs(r[r_c,"mass"] - lp.pi_plus.mass/1000.) < 1e-4
r["pi"] = r[r_c][pi_cut]

#Pi0s
pi0_cut = abs(r[r_c,"mass"] - lp.pi_0.mass/1000.) < 1e-4
r["pi0"] = r[r_c][pi0_cut]

In [ ]:
D = ak.cartesian({"pi": r["pi"], "pi0": r["pi0"]})

PDG_pi_m = lp.pi_plus.mass/1000.
PDG_pi0_m = lp.pi_0.mass/1000.
mass = kinematics.mass([D["pi"], D["pi0"]], [PDG_pi_m, PDG_pi0_m])

In [ ]:
PDG_D_m = lp.D_plus.mass/1000.
window = 0.6
mass_cut = abs(mass - PDG_D_m) < window
mass = mass[mass_cut]

In [ ]:
low = PDG_D_m - window
high = PDG_D_m + window
obs = zfit.Space('mD', limits=(low, high))

#Signal PDF
rand = random.randint(0,9999)
mu = zfit.Parameter(f"mu_{rand}", 1.86, low, high)
sigmaL = zfit.Parameter(f"sigmaL_{rand}", 0.03, 0., 0.5)
sigmaR = zfit.Parameter(f"sigmaR_{rand}", 0.05, 0., 0.5)
alphaL = zfit.Parameter(f"alphaL_{rand}", 1., 0., 5.)
nL = zfit.Parameter(f"nL_{rand}", 100., 0., 200.)
alphaR = zfit.Parameter(f"alphaR_{rand}", -1., -5., 0.)
nR = zfit.Parameter(f"nR_{rand}", 100., 0., 200.)
frac = zfit.Parameter(f"frac_{rand}", 0.4, 0., 1.)

pdf_sigL = zfit.pdf.CrystalBall(obs=obs, mu=mu, sigma=sigmaL, alpha=alphaL, n=nL)
pdf_sigR = zfit.pdf.CrystalBall(obs=obs, mu=mu, sigma=sigmaR, alpha=alphaR, n=nR)
pdf = zfit.pdf.SumPDF([pdf_sigL, pdf_sigR], frac)

In [ ]:
data_np = ak.to_numpy(ak.flatten(mass))[0:10000]
data = zfit.Data.from_numpy(obs=obs, array=data_np)

In [ ]:
nll = zfit.loss.UnbinnedNLL(model=pdf, data=data)
minimizer = zfit.minimize.Minuit(tolerance=1e-5)
result = minimizer.minimize(nll)
param_errors, _ = result.errors(method="minuit_minos")

print("m(D) fit function minimum:", result.fmin)
print("m(D) fit converged:", result.converged)
print("m(D) fit full minimizer information:", result.info)

params = result.params
print(params)

In [ ]:
fig,ax = plt.subplots(figsize=(10,8))
lower, upper = obs.limits
low = lower[-1][0]
high = upper[0][0]
bin_width = 0.01
bins = int(float(high - low)/bin_width)
counts, bin_edges = np.histogram(data_np, bins, range=(low,high))
bin_centres = (bin_edges[:-1] + bin_edges[1:])/2.
err = np.sqrt(counts)
plt.errorbar(bin_centres, counts, yerr=err, fmt='o', markersize=3, color='black')
x_plot = np.linspace(low, high, num=1000)
y_plot_tot = zfit.run(pdf.pdf(x_plot, norm_range=obs))
plt.plot(x_plot, y_plot_tot*len(data_np)/bins*obs.area(), color='navy', linewidth=2.5, alpha=0.6)
ax.tick_params(axis='both', which='major', labelsize=25)
plt.ylabel("Candidates / (%.3f GeV/$c^2$)" % bin_width,fontsize=25)
plt.xlabel("$m(\pi^\pm \pi^0)$ [GeV/$c^2$]",fontsize=25)
plt.xlim(low,high)
ymin, ymax = plt.ylim()
plt.ylim(0,1.05*ymax)
plt.text(2.,0.9*ymax,"\\textbf{FCC-ee}",fontsize=20)
plt.text(2.,0.82*ymax,"$Z^0 \\to c \\bar{c}$ sim.",fontsize=20)
plt.text(2.,0.74*ymax,"$D^\pm \\to \pi^\pm \pi^0$ excl.",fontsize=20)
plt.tight_layout()
fig.savefig(loc.PLOTS+f"/{file_type}_D_M_fit.pdf")

In [ ]:
#Number of Z's produced per year (4) per experiment (2)
N_Z = 3e12 * 0.25 * 0.5
#Number of c quarks produced
N_c = 2 * 0.1203 * N_Z
#Number of D -> pi pi0 produced (assume 43% production, same as for B mesons)
N_D2PiPi0 = N_c * 0.43 * 1.247e-3

print("N expected per year per exp = %.3e" % N_D2PiPi0)